# Transformer Encoder

지금까지 구현한 내용을 살펴보기 위해 전체 모델의 흐름을 다시 한 번 확인해 보죠.

- Image -> \[ Image_Embedding \] -> embedded tensor
- embedded tensor -> \[ Transformer Encoder \] -> extracted features
- extracted features -> \[ MLP Head \] -> Class probability


\[ MLP Head \] 에 대한 내용은 다음 장에서 다루도록 하고,

\[ Transformer Encoder \] 의 구현을 위한 구성을 fig 1 과 함께 sudocode 로 확인해 보죠.

<center>

<figure>
    <img src="./img/TransformerEncoder.png" alt="Transformer Encoder" width="20%" height="20%">
</figure>

<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    "fig 1: Transformer Encoder"
</figcaption>
  
</center>


>```json
>"Transformer Encoder":
>{
>  "Transformer Block": {
>      Multi-Head Attention
>      Layer Norm
>      Residual Connections
>      FeedForward Network
>  } "L times"
>}
>```

\[ Transformer Encoder \] 는 \[ Transformer Block \] N 개로 쌓여 있는 구조입니다. 

\[ Transformer Block \] 은 Multi-Head Attention, Layer Norm, \
Residual Connections, FeedForward Network 로 구성되어 있는것을 확인할 수 있습니다. 

이전 장에서는 \[ Transformer Block \] 에 요소인 Multi-Head Attention 을 구성해 보았습니다. 

이번 장에서는 \[ Transformer Block \] 에 필요한 모든 요소를 구성하여 \[ Transformer Encoder \] 를 완성하는 것을 목표로 합시다. 

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

import numpy as np
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
from collections import OrderedDict
from typing import Optional

from utils.vit_utils import Image_Embedding # 이전 장의 image embedding
from utils.vit_utils import Multi_Head_Attention # 이전 장의 Multi-Head Attention

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
ims = torch.Tensor(np.load('./resources/test_images.npy', allow_pickle=False))
ims = rearrange(ims, 'b h w c -> b c h w')
print(type(ims), ims.shape)

<class 'torch.Tensor'> torch.Size([6, 3, 96, 96])


---------

## Residual Connection
가장 먼저, Residual Connection 먼저 구성해 보죠.

In [3]:
class ResidualConnection(nn.Module):
    def __init__(self, layer):
        super().__init__()
        self.layer = layer
    
    def forward(self, x):
        temp_x = x
        x = self.layer(x)
        return x + temp_x

fig 1 에서 표현 되듯, MHA 와 MLP(FeedForward Network) 에 각각 적용될 수 있도록\
`__init__()` 함수에서 layer 를 인수로 받아 구성한 모습입니다.

---------

## FeedForward Network (MLP)

다음으로 FeedForward Network 를 구성해 봅시다.

In [4]:
class FeedForward(nn.Module):
    def __init__(self, 
                 embedding_size: int,
                 expansion: int = 4, 
                 dropout: float = 0.):
        super(FeedForward, self).__init__()

        self.ff_layer = nn.Sequential(
            nn.Linear(embedding_size, expansion * embedding_size),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(expansion * embedding_size, embedding_size),
        )
    def forward(self, x):
        return self.ff_layer(x)

입력으로 들어온 `embedding_size` 를 기준으로, \
expansion 배수만큼 parameter 를 키웠다가 복원하는 모습입니다. 

------
## Transformer Block

In [13]:
class Transformer_Block(nn.Module):
    def __init__(self, 
                 embedding_size: int = 768,
                 dropout: float = 0.,
                 forward_expansion: int = 4,
                 forward_dropout: float = 0,
                 **kwargs):
        super(Transformer_Block, self).__init__()
        self.norm_mha = nn.Sequential(
            ResidualConnection(
                nn.Sequential(
                    nn.LayerNorm(embedding_size),
                    Multi_Head_Attention(embedding_size, **kwargs),
                    nn.Dropout(dropout)
                    )
                )
            )
        self.norm_ff = nn.Sequential(
            ResidualConnection(
                nn.Sequential(
                    nn.LayerNorm(embedding_size),
                    FeedForward(embedding_size, forward_expansion, forward_dropout),
                    nn.Dropout(dropout)
                )
            )
        )

    def forward(self, x):
        x = self.norm_mha(x)
        return self.norm_ff(x)

In [14]:
class TransformerEncoder(nn.Module):
    def __init__(self, depth: int = 12, **kwargs):
        super(TransformerEncoder, self).__init__()
        self.multi_encoder_layer = nn.Sequential(*[Transformer_Block(**kwargs) for _ in range(depth)])    
        
    def forward(self, x):
        return self.multi_encoder_layer(x)


In [17]:
image_embedding = Image_Embedding(image_size = ims.shape, patch_size=16).to(device)
embedded_tensor = image_embedding(ims.to(device))

Vit = TransformerEncoder(embedding_size = 768, num_heads = 8).to(device)
summary(Vit, embedded_tensor.shape[1:])



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1              [-1, 37, 768]           1,536
            Linear-2              [-1, 37, 768]         590,592
            Linear-3              [-1, 37, 768]         590,592
            Linear-4              [-1, 37, 768]         590,592
            Linear-5              [-1, 37, 768]         590,592
Multi_Head_Attention-6              [-1, 37, 768]               0
           Dropout-7              [-1, 37, 768]               0
ResidualConnection-8              [-1, 37, 768]               0
         LayerNorm-9              [-1, 37, 768]           1,536
           Linear-10             [-1, 37, 3072]       2,362,368
             GELU-11             [-1, 37, 3072]               0
          Dropout-12             [-1, 37, 3072]               0
           Linear-13              [-1, 37, 768]       2,360,064
      FeedForward-14              [-1